In [1]:
import csv
import time
from pywifi import PyWiFi, const
def trilateration(nodes, distances):
    def equations(point):
        x, y, z = point
        return [((x - nx)**2 + (y - ny)**2 + (z - nz)**2 - d**2) for [nx, ny, nz], d in zip(nodes, distances)]
    initial_guess = [0, 0, 0]
    result = least_squares(equations, initial_guess)
    return result.x

In [2]:
def read_estimated_positions(filename="estimated_positions.csv"):
    with open(filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        positions = {row['BSSID']: [float(row['X Position']), float(row['Y Position']), float(row['Z Position'])] for row in reader}
    return positions

estimated_positions = read_estimated_positions()

In [3]:
def scan_wifi_for_trilateration():
    wifi = PyWiFi()
    iface = wifi.interfaces()[0]

    iface.scan()
    time.sleep(3)  # Wait for the scan to complete

    results = iface.scan_results()
    scanned_networks = {}

    for network in results:
        scanned_networks[network.bssid] = abs(int(network.signal))  # Signal strength as positive value

    return scanned_networks

scanned_networks = scan_wifi_for_trilateration()

In [4]:
def match_networks_with_positions(scanned_networks, estimated_positions):
    nodes = []
    distances = []

    for bssid, signal_strength in scanned_networks.items():
        if bssid in estimated_positions:
            nodes.append(estimated_positions[bssid])
            distances.append(signal_strength)  # Directly use signal strength as proxy for distance

    return nodes, distances

nodes, distances = match_networks_with_positions(scanned_networks, estimated_positions)

In [5]:
import csv
import time
from pywifi import PyWiFi, const
from scipy.optimize import least_squares

# Other functions (read_estimated_positions, scan_wifi_for_trilateration, match_networks_with_positions, trilateration) go here

def main():
    estimated_positions = read_estimated_positions()
    all_estimated_current_positions = []

    while True:
        user_input = input("Press Enter to scan for WiFi and estimate position (type 'quit' and press Enter to exit)...")
        if user_input.lower() == 'quit':
            break

        scanned_networks = scan_wifi_for_trilateration()
        nodes, distances = match_networks_with_positions(scanned_networks, estimated_positions)
        
        if nodes and distances:
            current_position = trilateration(nodes, distances)
            print(f"Estimated current position: {current_position}")
            all_estimated_current_positions.append(current_position)
        else:
            print("Not enough data for trilateration.")

    # Save the results to a CSV file
    with open('current_positions.csv', 'w', newline='') as csvfile:
        fieldnames = ['X Position', 'Y Position', 'Z Position']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for position in all_estimated_current_positions:
            writer.writerow({'X Position': position[0], 'Y Position': position[1], 'Z Position': position[2]})

    print("Current position estimates saved to 'current_positions.csv'.")

if __name__ == "__main__":
    main()

Press Enter to scan for WiFi and estimate position (type 'quit' and press Enter to exit)...
Estimated current position: [  0.          63.78036041 -29.03402782]
Press Enter to scan for WiFi and estimate position (type 'quit' and press Enter to exit)...
Estimated current position: [  0.          64.96202839 -35.19963263]
Press Enter to scan for WiFi and estimate position (type 'quit' and press Enter to exit)...
Estimated current position: [  0.          64.40781239 -33.39764969]
Press Enter to scan for WiFi and estimate position (type 'quit' and press Enter to exit)...quit


PermissionError: [Errno 13] Permission denied: 'current_positions.csv'